## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup

import pandas as pd
import requests
import gmaps

# Import API key
from config import gkey

# Configure gmaps API key
gmaps.configure(api_key=gkey)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Souillac,-20.52,57.52,69.80,68,40,9.17,MU,scattered clouds
1,1,Kovur,14.48,79.98,76.17,76,48,3.98,IN,scattered clouds
2,2,Atuona,-9.80,-139.03,77.47,79,68,15.12,PF,light rain
3,3,Bethel,41.37,-73.41,52.00,93,90,4.70,US,mist
4,4,Upernavik,72.79,-56.15,15.85,92,97,3.89,GL,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input('Please select minimum temperature '))
max_temp = float(input('Please select maximum temperature '))

Please select minimum temperature 35
Please select maximum temperature 79


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# res_df = city_data_df.loc[(city_data_df['Max Temp'] <maxx) & (city_data_df['Max Temp'] >minn)]
# res_df

res_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

In [5]:
# 4a. Determine if there are any empty rows.
res_df.isnull().sum()


City_ID                0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Country                5
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
res_df.dropna()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Souillac,-20.52,57.52,69.80,68,40,9.17,MU,scattered clouds
1,1,Kovur,14.48,79.98,76.17,76,48,3.98,IN,scattered clouds
2,2,Atuona,-9.80,-139.03,77.47,79,68,15.12,PF,light rain
3,3,Bethel,41.37,-73.41,52.00,93,90,4.70,US,mist
5,5,Busselton,-33.65,115.33,57.99,77,84,5.48,AU,broken clouds
...,...,...,...,...,...,...,...,...,...,...
664,664,Luena,-11.78,19.92,65.34,63,0,3.71,AO,clear sky
665,665,Pringsewu,-5.36,104.97,73.80,85,69,4.09,ID,broken clouds
666,666,Bocas Del Toro,9.33,-82.25,64.99,100,100,11.05,PA,moderate rain
667,667,Ossipee,43.69,-71.12,46.00,94,100,5.91,US,moderate rain


In [ ]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = city_data_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

In [ ]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gkey
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
   # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

In [ ]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna()


In [ ]:
hotel_df.isnull().sum()

In [ ]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search_Database.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [ ]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

hotel_df.count()

In [ ]:
# 11a. Add a marker layer for each city to the map. 
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
hotel_name = hotel_df["Hotel Name"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig